In [ ]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from scipy import stats
from scipy.stats import norm

In [ ]:
train = pd.read_csv("../input/estyle-community-competition-2025/train.csv")
test = pd.read_csv("../input/estyle-community-competition-2025/test.csv")

In [ ]:
# データの基本情報を確認
print("=== Train Shape ===")
print(train.shape)
print("\n=== Test Shape ===")
print(test.shape)
print("\n=== Train Columns ===")
print(train.columns.tolist())
print("\n=== Missing Values ===")
print(train.isnull().sum()[train.isnull().sum() > 0].sort_values(ascending=False))

# 欠損値処理と特徴量エンジニアリング

戦略ドキュメントに基づいて、以下の手順で処理を実施します：
1. 欠損値処理（MNAR/MAR に応じた補完）
2. 欠損フラグの作成
3. 派生特徴量の作成
4. カテゴリカル変数のエンコーディング

In [ ]:
# Train/Test統合してデータ処理（後で分割）
# まずはTrainのIDとTradePriceを保存
train_ids = train['Id'].copy()
train_target = train['TradePrice'].copy()
test_ids = test['Id'].copy()

# TrainとTestを結合（TradePriceは除く）
train_features = train.drop('TradePrice', axis=1)
all_data = pd.concat([train_features, test], axis=0, ignore_index=True)

print(f"Combined data shape: {all_data.shape}")
print(f"Train samples: {len(train_features)}, Test samples: {len(test)}")

## ステップ1: 欠損フラグの作成（MNAR変数用）

In [ ]:
# 建物関連の欠損フラグ（MNAR: 建物なし物件）
all_data['HasBuilding'] = all_data['TotalFloorArea'].notnull().astype(int)
all_data['HasFloorPlan'] = all_data['FloorPlan'].notnull().astype(int)
all_data['HasStructure'] = all_data['Structure'].notnull().astype(int)
all_data['HasBuildingYear'] = all_data['BuildingYear'].notnull().astype(int)

# 道路関連の欠損フラグ（MNAR: 道路なし物件）
all_data['HasRoadAccess'] = all_data['Breadth'].notnull().astype(int)
all_data['HasRoadClassification'] = all_data['Classification'].notnull().astype(int)

# 土地関連の欠損フラグ（MAR: 測定困難）
all_data['HasFrontage'] = all_data['Frontage'].notnull().astype(int)
all_data['HasLandShape'] = all_data['LandShape'].notnull().astype(int)
all_data['HasDirection'] = all_data['Direction'].notnull().astype(int)

# 駅情報の欠損フラグ
all_data['HasStation'] = all_data['NearestStation'].notnull().astype(int)

# 備考の欠損フラグ（備考あり物件は価格低め傾向）
all_data['HasRemarks'] = all_data['Remarks'].notnull().astype(int)

print("欠損フラグ作成完了")
print(f"新規作成フラグ: {['HasBuilding', 'HasFloorPlan', 'HasStructure', 'HasBuildingYear', 'HasRoadAccess', 'HasRoadClassification', 'HasFrontage', 'HasLandShape', 'HasDirection', 'HasStation', 'HasRemarks']}")

## ステップ2: 欠損値の補完

In [ ]:
# 2-1. 駅情報の補完
# NearestStation: 欠損は"No Station"で補完
all_data['NearestStation'] = all_data['NearestStation'].fillna('No Station')

# MinTimeToNearestStation: Type別の中央値で補完
all_data['MinTimeToNearestStation'] = all_data.groupby('Type')['MinTimeToNearestStation'].transform(
    lambda x: x.fillna(x.median())
)
# まだ欠損があれば（Typeごとに全て欠損の場合）全体中央値で補完
all_data['MinTimeToNearestStation'] = all_data['MinTimeToNearestStation'].fillna(
    all_data['MinTimeToNearestStation'].median()
)

# MaxTimeToNearestStationとTimeToNearestStationは削除（情報冗長）
all_data = all_data.drop(['MaxTimeToNearestStation', 'TimeToNearestStation'], axis=1)

print("駅情報の補完完了")

In [ ]:
# 2-2. 地域情報の補完
# Region: 欠損は'Unknown'で補完（林地・農地など地域区分なし物件）
all_data['Region'] = all_data['Region'].fillna('Unknown')

print("地域情報の補完完了")

In [ ]:
# 2-3. 建物情報の補完
# FloorPlan: 欠損は"No Building"で補完
all_data['FloorPlan'] = all_data['FloorPlan'].fillna('No Building')

# TotalFloorArea: 欠損は0で補完（建物なし）
all_data['TotalFloorArea'] = all_data['TotalFloorArea'].fillna(0)

# BuildingYear: 欠損は0で補完（建物なし/築年不詳）
all_data['BuildingYear'] = all_data['BuildingYear'].fillna(0)

# Structure: 欠損は"No Building"で補完
all_data['Structure'] = all_data['Structure'].fillna('No Building')

# Use: 欠損は"Vacant Land"で補完（空地）
all_data['Use'] = all_data['Use'].fillna('Vacant Land')

# Purpose: 欠損率65%と高いため削除
all_data = all_data.drop('Purpose', axis=1)

# Renovation: 欠損は"Unknown"で補完
all_data['Renovation'] = all_data['Renovation'].fillna('Unknown')

print("建物情報の補完完了")

In [ ]:
# 2-4. 土地情報の補完
# LandShape: 欠損は'Unknown'で補完
all_data['LandShape'] = all_data['LandShape'].fillna('Unknown')

# Frontage: Type別の中央値で補完（測定困難なケース）
all_data['Frontage'] = all_data.groupby('Type')['Frontage'].transform(
    lambda x: x.fillna(x.median())
)
# 残りは全体中央値で補完
all_data['Frontage'] = all_data['Frontage'].fillna(all_data['Frontage'].median())

# Direction: 欠損は'Unknown'で補完
all_data['Direction'] = all_data['Direction'].fillna('Unknown')

print("土地情報の補完完了")

In [ ]:
# 2-5. 道路情報の補完
# Classification: 欠損は"No Road"で補完（道路なし）
all_data['Classification'] = all_data['Classification'].fillna('No Road')

# Breadth: 欠損は0で補完（道路なし）
all_data['Breadth'] = all_data['Breadth'].fillna(0)

print("道路情報の補完完了")

In [ ]:
# 2-6. 都市計画・法規制情報の補完
# CityPlanning: 欠損は"Outside City Planning"で補完
all_data['CityPlanning'] = all_data['CityPlanning'].fillna('Outside City Planning')

# CoverageRatio: Type別の中央値で補完
all_data['CoverageRatio'] = all_data.groupby('Type')['CoverageRatio'].transform(
    lambda x: x.fillna(x.median())
)
all_data['CoverageRatio'] = all_data['CoverageRatio'].fillna(all_data['CoverageRatio'].median())

# FloorAreaRatio: Type別の中央値で補完
all_data['FloorAreaRatio'] = all_data.groupby('Type')['FloorAreaRatio'].transform(
    lambda x: x.fillna(x.median())
)
all_data['FloorAreaRatio'] = all_data['FloorAreaRatio'].fillna(all_data['FloorAreaRatio'].median())

print("都市計画情報の補完完了")

In [ ]:
# 2-7. その他の補完
# DistrictName: 欠損が0.05%と少ないが念のため"Unknown"で補完
all_data['DistrictName'] = all_data['DistrictName'].fillna('Unknown')

# Remarks: 情報量少ないため削除（HasRemarksフラグで代用）
all_data = all_data.drop('Remarks', axis=1)

print("その他の補完完了")
print(f"\n=== 残存する欠損値 ===")
print(all_data.isnull().sum()[all_data.isnull().sum() > 0])

## ステップ3: 派生特徴量の作成

In [ ]:
# 3-1. 欠損パターン集約特徴
# 建物情報の欠損数（0〜4）
building_cols = ['FloorPlan', 'TotalFloorArea', 'BuildingYear', 'Structure']
all_data['MissingBuildingInfo'] = (
    (1 - all_data['HasFloorPlan']) + 
    (1 - all_data['HasBuilding']) + 
    (1 - all_data['HasBuildingYear']) + 
    (1 - all_data['HasStructure'])
)

# 道路情報の欠損数（0〜4）
all_data['MissingRoadInfo'] = (
    (1 - all_data['HasRoadAccess']) + 
    (1 - all_data['HasRoadClassification']) + 
    (1 - all_data['HasFrontage']) + 
    (1 - all_data['HasDirection'])
)

print("欠損パターン集約特徴作成完了")
print(f"MissingBuildingInfo分布:\n{all_data['MissingBuildingInfo'].value_counts().sort_index()}")
print(f"\nMissingRoadInfo分布:\n{all_data['MissingRoadInfo'].value_counts().sort_index()}")

In [ ]:
# 3-2. 築年数の計算
# BuildingAge = Year - BuildingYear
all_data['BuildingAge'] = all_data['Year'] - all_data['BuildingYear']
# BuildingYear=0（欠損補完値）の場合は築年数を-1に設定
all_data.loc[all_data['BuildingYear'] == 0, 'BuildingAge'] = -1

print("築年数特徴作成完了")
print(f"BuildingAge統計:\n{all_data['BuildingAge'].describe()}")
print(f"\nBuildingAge=-1（建物なし）の件数: {(all_data['BuildingAge'] == -1).sum()}")

In [ ]:
# 3-3. 面積関連特徴
# 延床面積率（容積率の実現値）= TotalFloorArea / Area
all_data['FloorAreaRatioActual'] = all_data['TotalFloorArea'] / all_data['Area']
# 無限大やNaNを0に置換
all_data['FloorAreaRatioActual'] = all_data['FloorAreaRatioActual'].replace([np.inf, -np.inf], 0).fillna(0)

# 容積率余裕度 = FloorAreaRatioActual / FloorAreaRatio（法定容積率）
# 0除算を避けるため、FloorAreaRatio > 0 の場合のみ計算
all_data['FloorAreaRatioUsage'] = 0.0
mask = all_data['FloorAreaRatio'] > 0
all_data.loc[mask, 'FloorAreaRatioUsage'] = (
    all_data.loc[mask, 'FloorAreaRatioActual'] / all_data.loc[mask, 'FloorAreaRatio']
)

print("面積関連特徴作成完了")
print(f"FloorAreaRatioActual統計:\n{all_data['FloorAreaRatioActual'].describe()}")

In [ ]:
# 3-4. 四半期のOne-Hot化（時間的特徴）
all_data['Quarter_Q1'] = (all_data['Quarter'] == 1).astype(int)
all_data['Quarter_Q2'] = (all_data['Quarter'] == 2).astype(int)
all_data['Quarter_Q3'] = (all_data['Quarter'] == 3).astype(int)
all_data['Quarter_Q4'] = (all_data['Quarter'] == 4).astype(int)

print("時間的特徴作成完了")

## ステップ4: カテゴリカル変数のエンコーディング

In [ ]:
# カテゴリカル変数の分類
# 低カーディナリティ（One-Hotエンコーディング）
low_cardinality_cols = [
    'Type', 'Prefecture', 'Structure', 'LandShape', 
    'Direction', 'Classification', 'Renovation', 'Region'
]

# 中カーディナリティ（Label Encodingまたはターゲットエンコーディング）
medium_cardinality_cols = ['CityPlanning', 'Municipality', 'Use', 'FloorPlan']

# 高カーディナリティ（ターゲットエンコーディング推奨）
high_cardinality_cols = ['NearestStation', 'DistrictName']

print("カテゴリカル変数の分類完了")
print(f"低カーディナリティ: {low_cardinality_cols}")
print(f"中カーディナリティ: {medium_cardinality_cols}")
print(f"高カーディナリティ: {high_cardinality_cols}")

In [ ]:
# 4-1. 低カーディナリティ変数のOne-Hotエンコーディング
for col in low_cardinality_cols:
    if col in all_data.columns:
        # One-Hotエンコーディング（最初のカテゴリをdropして多重共線性を回避）
        dummies = pd.get_dummies(all_data[col], prefix=col, drop_first=True, dtype=int)
        all_data = pd.concat([all_data, dummies], axis=1)
        print(f"{col}: {all_data[col].nunique()}カテゴリ → {len(dummies.columns)}次元")

print("\nOne-Hotエンコーディング完了")

In [ ]:
# 4-2. 中・高カーディナリティ変数のLabel Encoding
# LightGBM用にカテゴリをそのまま整数化
from sklearn.preprocessing import LabelEncoder

label_encoding_cols = medium_cardinality_cols + high_cardinality_cols

for col in label_encoding_cols:
    if col in all_data.columns:
        le = LabelEncoder()
        all_data[f'{col}_Encoded'] = le.fit_transform(all_data[col].astype(str))
        n_categories = all_data[f'{col}_Encoded'].nunique()
        print(f"{col}: {n_categories}カテゴリをLabel Encoding")

print("\nLabel Encoding完了")

## ステップ5: 最終データセットの準備

In [ ]:
# 不要なカラムを削除（元のカテゴリカル変数、ID、Quarterは削除）
# 元のカテゴリカル変数を削除（エンコード済み）
drop_cols = (
    ['Id', 'Quarter'] + 
    low_cardinality_cols + 
    medium_cardinality_cols + 
    high_cardinality_cols
)

# 存在するカラムのみ削除
drop_cols_existing = [col for col in drop_cols if col in all_data.columns]
all_data_processed = all_data.drop(drop_cols_existing, axis=1)

print(f"削除したカラム数: {len(drop_cols_existing)}")
print(f"削除したカラム: {drop_cols_existing[:10]}...")
print(f"\n処理後のデータ形状: {all_data_processed.shape}")
print(f"カラム数: {all_data_processed.shape[1]}")

In [ ]:
# Train/Testに分割
train_processed = all_data_processed.iloc[:len(train_features)].copy()
test_processed = all_data_processed.iloc[len(train_features):].copy()

# IDとターゲットを追加
train_processed['Id'] = train_ids.values
train_processed['TradePrice'] = train_target.values
test_processed['Id'] = test_ids.values

print(f"Train processed shape: {train_processed.shape}")
print(f"Test processed shape: {test_processed.shape}")
print(f"\n=== 処理完了サマリー ===")
print(f"元のTrain特徴量数: {train_features.shape[1] - 1}")  # Idを除く
print(f"処理後の特徴量数: {train_processed.shape[1] - 2}")  # Id, TradePriceを除く
print(f"追加された特徴量数: {(train_processed.shape[1] - 2) - (train_features.shape[1] - 1)}")

In [ ]:
# 処理済みデータの保存
train_processed.to_csv('../output/train_processed.csv', index=False)
test_processed.to_csv('../output/test_processed.csv', index=False)

print("処理済みデータを保存しました:")
print("- ../output/train_processed.csv")
print("- ../output/test_processed.csv")

In [ ]:
# 最終的な特徴量リストの確認
feature_cols = [col for col in train_processed.columns if col not in ['Id', 'TradePrice']]

print(f"\n=== 特徴量サマリー ===")
print(f"総特徴量数: {len(feature_cols)}")
print(f"\n特徴量の内訳:")
print(f"  - 欠損フラグ: 11個")
print(f"  - 派生特徴（欠損パターン集約、築年数、面積関連等）: 7個")
print(f"  - 四半期One-Hot: 4個")
print(f"  - カテゴリOne-Hot: 74個")
print(f"  - カテゴリLabel Encoding: 6個")
print(f"  - 元の数値特徴: 12個程度")
print(f"\n主要な特徴量（最初の20個）:")
print(feature_cols[:20])